Setup

In [1]:
from tensorflow_docs.vis import embed

from tensorflow import keras
from keras import layers

import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import imageio
import cv2
import os

Data preparation

In [2]:
#INDEX 5
MAX_SEQ_LENGTH_TRANSFORM = 40
NUM_FEATURES_TRANSFORM = 1024

#INDEX 15 (DENSE)
NUM_FEATURES_DENSE = 2048

#INDEX 10 (GRU) y 11 (LSTM)
MAX_SEQ_LENGTH_RNN = 40
NUM_FEATURES_RNN = 2048

IMG_SIZE = 128

CLASSES = 4

CNN (DenseNet121 or InceptionV3)

In [3]:
center_crop_layer = layers.CenterCrop(IMG_SIZE, IMG_SIZE)

def crop_center(frame):
    cropped = center_crop_layer(frame[None, ...])
    cropped = cropped.numpy().squeeze()
    return cropped

def IsBlack(frame, y, x):
    height, width, _ = frame.shape
    cond = True
    r = 2
    i = -r
    j = -r
    while i < r and i + y < height and i + y > 0 and cond:
        j = 0
        while j < r and j + x < width and j + x > 0 and cond:
            if frame[y+i, x+j].any() != 0:
                cond = False
            j = j + 1
        i = i + 1
              
    return cond

def get_dimension(frame):
    height, width, _ = frame.shape

    halfWidth = int(width / 2)
    halfHeight = int(height / 2)

    up = 0
    down = height - 1
    left = halfWidth - 1
    right = halfWidth + 1

    while IsBlack(frame, up, halfWidth) and up < halfHeight:
            up = up + 1
    while IsBlack(frame, down, halfWidth) and down > halfHeight:
            down = down - 1

    while not IsBlack(frame, up + 5, left) and left > 0:
            left = left - 1
    while not IsBlack(frame, up + 5, right) and right < width:
            right = right + 1

    up = up - 1
    down = down + 1
    left = left - 1
    right = right + 1

    return right, up, left, down

def load_video(path, max_frames=0):

    cap = cv2.VideoCapture(path)
    frames = []

    try:
        ret, frame = cap.read()
        right, up, left, down = get_dimension(frame)
        while True:

            if not ret:
                break

            frame = frame[up:down, left:right]
            frame = crop_center(frame)
            frame = frame[:, :, [2, 1, 0]]
            frames.append(frame)

            if len(frames) == max_frames:
                break            
            ret, frame = cap.read()
    finally:
        cap.release()
    return np.array(frames)

In [4]:
def build_feature_extractor():
    feature_extractor = keras.applications.DenseNet121(
        weights="imagenet",
        include_top=False,
        pooling="avg",
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
    )
    preprocess_input = keras.applications.densenet.preprocess_input

    inputs = keras.Input((IMG_SIZE, IMG_SIZE, 3))
    preprocessed = preprocess_input(inputs)

    outputs = feature_extractor(preprocessed)
    return keras.Model(inputs, outputs, name="feature_extractor")


feature_extractor_TRANSFORMER = build_feature_extractor()

In [5]:
def build_feature_extractor():
    feature_extractor = keras.applications.InceptionV3(
        weights="imagenet",
        include_top=False,
        pooling="avg",
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
    )
    preprocess_input = keras.applications.inception_v3.preprocess_input

    inputs = keras.Input((IMG_SIZE, IMG_SIZE, 3))
    preprocessed = preprocess_input(inputs)

    outputs = feature_extractor(preprocessed)
    return keras.Model(inputs, outputs, name="feature_extractor")


feature_extractor_RNN = build_feature_extractor()

TRANSFORMER

In [6]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim
        )
        self.sequence_length = sequence_length
        self.output_dim = output_dim

    def call(self, inputs):
        # The inputs are of shape: `(batch_size, frames, num_features)`
        length = tf.shape(inputs)[1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_positions = self.position_embeddings(positions)
        return inputs + embedded_positions

    def compute_mask(self, inputs, mask=None):
        mask = tf.reduce_any(tf.cast(inputs, "bool"), axis=-1)
        return mask

class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim, dropout=0.3
        )
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation=tf.nn.gelu), layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()

    def call(self, inputs, mask=None):
        if mask is not None:
            mask = mask[:, tf.newaxis, :]

        attention_output = self.attention(inputs, inputs, attention_mask=mask)
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)
    
def get_compiled_model():
    sequence_length = MAX_SEQ_LENGTH_TRANSFORM
    embed_dim = NUM_FEATURES_TRANSFORM
    dense_dim = 4
    num_heads = 1

    inputs = keras.Input(shape=(None, None))
    x = PositionalEmbedding(
        sequence_length, embed_dim, name="frame_position_embedding"
    )(inputs)
    x = TransformerEncoder(embed_dim, dense_dim, num_heads, name="transformer_layer")(x)
    x = layers.GlobalMaxPooling1D()(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(CLASSES, activation="softmax")(x)
    model = keras.Model(inputs, outputs)

    model.compile(
        optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
    )
    return model


def run_experiment():
    filepath = f"Model/Transformer/video_classifier"
    
    model = get_compiled_model()
    model.load_weights(filepath)
    model.summary()

    return model

TRANSFORMER_model = run_experiment()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, None, None)]      0         
                                                                 
 frame_position_embedding (P  (None, None, 1024)       40960     
 ositionalEmbedding)                                             
                                                                 
 transformer_layer (Transfor  (None, None, 1024)       4211716   
 merEncoder)                                                     
                                                                 
 global_max_pooling1d (Globa  (None, 1024)             0         
 lMaxPooling1D)                                                  
                                                                 
 dropout (Dropout)           (None, 1024)              0         
                                                             

GRU

In [7]:
def get_sequence_model():
    
    frame_features_input = keras.Input((MAX_SEQ_LENGTH_RNN, NUM_FEATURES_RNN))
    mask_input = keras.Input((MAX_SEQ_LENGTH_RNN,), dtype="bool")

    # Refer to the following tutorial to understand the significance of using `mask`:
    # https://keras.io/api/layers/recurrent_layers/gru/
    x = keras.layers.GRU(16, return_sequences=True)(
        frame_features_input, mask=mask_input
    )
    x = keras.layers.GRU(8)(x)
    x = keras.layers.Dropout(0.4)(x)
    x = keras.layers.Dense(8, activation="relu")(x)
    output = keras.layers.Dense(CLASSES, activation="softmax")(x)

    rnn_model = keras.Model([frame_features_input, mask_input], output)

    rnn_model.compile(
        loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
    )
    return rnn_model

def run_experiment():
    filepath = f"Model/GRU/video_classifier"

    seq_model = get_sequence_model()
    seq_model.load_weights(filepath)
    seq_model.summary()

    return seq_model

GRU_model = run_experiment()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_6 (InputLayer)           [(None, 40, 2048)]   0           []                               
                                                                                                  
 input_7 (InputLayer)           [(None, 40)]         0           []                               
                                                                                                  
 gru (GRU)                      (None, 40, 16)       99168       ['input_6[0][0]',                
                                                                  'input_7[0][0]']                
                                                                                                  
 gru_1 (GRU)                    (None, 8)            624         ['gru[0][0]']              

LSTM

In [8]:
def get_sequence_model():
    frame_features_input = keras.Input((MAX_SEQ_LENGTH_RNN, NUM_FEATURES_RNN))
    mask_input = keras.Input((MAX_SEQ_LENGTH_RNN,), dtype="bool")

    x = keras.layers.LSTM(16, return_sequences=True)(
        frame_features_input, mask=mask_input
    )
    x = keras.layers.LSTM(8)(x)
    x = keras.layers.Dropout(0.3)(x)
    x = keras.layers.Dense(8, activation="relu")(x)
    output = keras.layers.Dense(CLASSES, activation="softmax")(x)

    rnn_model = keras.Model([frame_features_input, mask_input], output)

    rnn_model.compile(
        loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
    )
    return rnn_model

def run_experiment():
    filepath = f"Model/LSTM/video_classifier"

    seq_model = get_sequence_model()
    seq_model.load_weights(filepath)
    seq_model.summary()

    return seq_model

LSTM_model = run_experiment()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_8 (InputLayer)           [(None, 40, 2048)]   0           []                               
                                                                                                  
 input_9 (InputLayer)           [(None, 40)]         0           []                               
                                                                                                  
 lstm (LSTM)                    (None, 40, 16)       132160      ['input_8[0][0]',                
                                                                  'input_9[0][0]']                
                                                                                                  
 lstm_1 (LSTM)                  (None, 8)            800         ['lstm[0][0]']             

Inference

In [9]:
def get_type(index):
    if (index == 0):
        return feature_extractor_TRANSFORMER, MAX_SEQ_LENGTH_TRANSFORM, NUM_FEATURES_TRANSFORM
    if (index == 1 or index == 2):
        return feature_extractor_RNN, MAX_SEQ_LENGTH_RNN, NUM_FEATURES_RNN
    
def get_model(index):
    if (index == 0):
        return TRANSFORMER_model
    if (index == 1):
        return GRU_model
    if (index == 2):
        return LSTM_model
    
def get_model_name(index):
    if (index == 0):
        return "TRANSFORMER"
    if (index == 1):
        return "GRU"
    if (index == 2):
        return "LSTM"

In [12]:
def prepare_single_video(frames, index):
    feature_extractor, max_seq_length, num_features = get_type(index)
    frame_features = np.zeros(shape=(1, max_seq_length, num_features), dtype="float32")
    frame_mask = np.zeros(shape=(1, max_seq_length,), dtype="bool")

    if len(frames) < max_seq_length:
        diff = max_seq_length - len(frames)
        padding = np.zeros((diff, IMG_SIZE, IMG_SIZE, 3))
        frames = np.concatenate(frames, padding)

    frames = frames[None, ...]

    for i, batch in enumerate(frames):
        video_length = batch.shape[0]
        length = min(max_seq_length, video_length)
        for j in range(length):
            if np.mean(batch[j, :]) > 0.0:
                frame_features[i, j, :] = feature_extractor.predict(batch[None, j, :], verbose=0)
            else:
                frame_features[i, j, :] = 0.0
        frame_mask[i, :length] = 1

    return frame_features, frame_mask


def predict_action(path, index):

    model = get_model(index)

    class_vocab = np.arange(CLASSES)

    frames = load_video(path)
    
    frame_features, frame_mask = prepare_single_video(frames, index)

    if (index == 0):
        probabilities = model.predict(frame_features)[0]
    if (index == 1 or index == 2):    
        probabilities = model.predict([frame_features, frame_mask])[0]
        
    total_probabilities = ["" for x in range(CLASSES)]
    j = 0
    for i in np.argsort(probabilities)[::-1]:
        total_probabilities[j] = f"{class_vocab[i]}: {probabilities[i] * 100:5.2f}%"
        print(total_probabilities[j])
        j = j + 1
    return total_probabilities

def predict_actions(path):

    class_vocab = np.arange(CLASSES)

    frames = load_video(path)
    
    print(class_vocab)
    for i in range(4):
        
        if (i < 2):
            frame_features, frame_mask = prepare_single_video(frames, i)

        model = get_model(i)
        if (i == 0):
            probabilities = model.predict(frame_features)[0]
        if (i == 1 and i == 2):
            probabilities = model.predict([frame_features, frame_mask])[0]

        print(get_model_name(i))        
        print(probabilities)
        for i in np.argsort(probabilities)[::-1]:
            print(f"  {class_vocab[i]}: {probabilities[i] * 100:5.2f}%")

In [14]:
test_video = f"D://Facultad//PPS//Proyecto Ecografia//Noviembre2020//PAC16//POST.mov"

#print("TRANSFORMER:")
#predict_action(test_video, 0)
print("GRU:")
predict_action(test_video, 1)
print("LSTM:")
predict_action(test_video, 2)

TRANSFORMER:
1/1 [==============================] - 0s 19ms/step
3: 83.86%
0:  9.90%
1:  3.78%
2:  2.46%
GRU:
1/1 [==============================] - 0s 19ms/step
3: 56.23%
2: 21.21%
1: 14.22%
0:  8.34%
LSTM:
1/1 [==============================] - 0s 20ms/step
0: 85.77%
1:  9.14%
3:  3.67%
2:  1.42%


['0: 85.77%', '1:  9.14%', '3:  3.67%', '2:  1.42%']

In [26]:
test_video = f"D://Facultad//PPS//Proyecto Ecografia//Noviembre2020//PAC14//POST.mov"

predict_actions(test_video)

[0 1 2 3]
1/1 [==============================] - 0s 19ms/step
TRANSFORMER
[0.00584809 0.10766613 0.0112304  0.8752554 ]
  3: 87.53%
  1: 10.77%
  2:  1.12%
  0:  0.58%
GRU
[0.00584809 0.10766613 0.0112304  0.8752554 ]
  3: 87.53%
  1: 10.77%
  2:  1.12%
  0:  0.58%
LSTM
[0.00584809 0.10766613 0.0112304  0.8752554 ]
  3: 87.53%
  1: 10.77%
  2:  1.12%
  0:  0.58%
1/1 [==============================] - 0s 16ms/step
Dense
[0.03166831 0.6536658  0.01988611 0.29477978]
  1: 65.37%
  3: 29.48%
  0:  3.17%
  2:  1.99%


In [95]:
# Ruta del directorio que deseas leer
dir = f'D://Facultad//PPS//Proyecto Ecografia//Noviembre2020'
df = pd.DataFrame({'Direccion':[], 'Target Transformer':[], 'Target GRU':[], 'Target LSTM':[]})

def MostrarFuncion(df, directorio):

    for archivo in os.listdir(directorio):

        ruta_completa = os.path.join(directorio, archivo)
        
        if os.path.isdir(ruta_completa):
            df = MostrarFuncion(df, ruta_completa)
        if os.path.isfile(ruta_completa) and ruta_completa.split('.')[-1] == 'mov':
            nueva_fila = {
                'Direccion':ruta_completa, 
                'Target Transformer' : predict_action(ruta_completa, 0), 
                'Target GRU' : predict_action(ruta_completa, 1), 
                'Target LSTM' : predict_action(ruta_completa, 2)
                }
            df = df.append(nueva_fila, ignore_index=True)

    return df

display(df)
df = MostrarFuncion(df, dir)
df.to_csv('resultados.csv', index=False)

,Direccion,Target Transformer,Target GRU,Target LSTM


1/1 [==============================] - 0s 20ms/step
3: 93.56%
0:  3.06%
1:  2.12%
2:  1.27%
1/1 [==============================] - 0s 20ms/step
3: 56.54%
2: 21.94%
1: 12.69%
0:  8.84%
1/1 [==============================] - 0s 22ms/step
3: 68.90%
1: 22.90%
0:  6.21%
2:  1.99%


C:\Users\Francisco\AppData\Local\Temp\ipykernel_7888\1888577245.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(nueva_fila, ignore_index=True)


1/1 [==============================] - 0s 24ms/step
3: 95.47%
1:  3.28%
0:  0.85%
2:  0.40%
1/1 [==============================] - 0s 21ms/step
3: 69.31%
2: 20.94%
1:  6.98%
0:  2.77%
1/1 [==============================] - 0s 25ms/step
3: 68.96%
1: 22.67%
0:  6.28%
2:  2.08%


C:\Users\Francisco\AppData\Local\Temp\ipykernel_7888\1888577245.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(nueva_fila, ignore_index=True)


...

C:\Users\Francisco\AppData\Local\Temp\ipykernel_7888\1888577245.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(nueva_fila, ignore_index=True)


1/1 [==============================] - 0s 23ms/step
3: 51.92%
1: 47.09%
2:  0.90%
0:  0.09%
1/1 [==============================] - 0s 21ms/step
3: 68.78%
2: 21.34%
1:  6.85%
0:  3.03%
1/1 [==============================] - 0s 22ms/step
3: 36.79%
1: 22.85%
0: 20.98%
2: 19.38%


C:\Users\Francisco\AppData\Local\Temp\ipykernel_7888\1888577245.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(nueva_fila, ignore_index=True)


1/1 [==============================] - 0s 21ms/step
3: 92.98%
1:  4.66%
0:  1.65%
2:  0.71%
1/1 [==============================] - 0s 21ms/step
3: 50.68%
2: 21.02%
1: 14.43%
0: 13.87%
1/1 [==============================] - 0s 23ms/step
3: 33.40%
2: 23.37%
0: 22.56%
1: 20.68%


C:\Users\Francisco\AppData\Local\Temp\ipykernel_7888\1888577245.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(nueva_fila, ignore_index=True)


In [21]:
# Ruta del directorio que deseas leer
dir = f'D://Facultad//PPS//Proyecto Ecografia//Noviembre2020'
df = pd.DataFrame({'Direccion':[], 'Target DENSE':[]})

def MostrarFuncion(df, directorio):

    for archivo in os.listdir(directorio):

        ruta_completa = os.path.join(directorio, archivo)
        
        if os.path.isdir(ruta_completa):
            df = MostrarFuncion(df, ruta_completa)
        if os.path.isfile(ruta_completa) and ruta_completa.split('.')[-1] == 'mov':
            nueva_fila = {
                'Direccion':ruta_completa, 
                'Target DENSE' : predict_action(ruta_completa, 3),
                }
            df = df.append(nueva_fila, ignore_index=True)

    return df

display(df)
df = MostrarFuncion(df, dir)
df.to_csv('resultados_dense.csv', index=False)

,Direccion,Target DENSE


1/1 [==============================] - 0s 15ms/step
3: 78.03%
1: 14.69%
0:  7.22%
2:  0.06%


C:\Users\Francisco\AppData\Local\Temp\ipykernel_7812\2194254065.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(nueva_fila, ignore_index=True)


1/1 [==============================] - 0s 15ms/step
3: 100.00%
1:  0.00%
0:  0.00%
2:  0.00%


C:\Users\Francisco\AppData\Local\Temp\ipykernel_7812\2194254065.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(nueva_fila, ignore_index=True)


...

C:\Users\Francisco\AppData\Local\Temp\ipykernel_7812\2194254065.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(nueva_fila, ignore_index=True)


1/1 [==============================] - 0s 16ms/step
3: 98.04%
1:  1.14%
2:  0.73%
0:  0.09%


C:\Users\Francisco\AppData\Local\Temp\ipykernel_7812\2194254065.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(nueva_fila, ignore_index=True)


1/1 [==============================] - 0s 16ms/step
3: 89.29%
1:  9.09%
0:  1.52%
2:  0.11%


C:\Users\Francisco\AppData\Local\Temp\ipykernel_7812\2194254065.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(nueva_fila, ignore_index=True)
